# ARP Lab
## A Retirement Planning Laboratory

This package is a retirement modeling framework for exploring the sensitivity of retirement financial decisions. Strictly speaking, it is not a planning tool, but more an environment for exploring *what if* scenarios. It provides different realizations of a financial strategy. One can certainly have a savings plan, but due to the volatility of financial investments, it is impossible to have a certain asset earnings plan. That does not mean one cannot make decisions. This is where this tool fits it. Given your savings and spending desires, it can generate different future realizations of your strategy under different market assumptions, helping to better understand one's financial situation.

Copyright - Martin-D. Lacasse (2023)

Disclaimers: *I am not a financial planner. You make your own decisions. This program comes with no guarantee. Use at your own risk.*

## Introduction
This file is provided as an example to introduce you to ARP Lab.

I assume that you have some familiarity with using a jupyter notebook or jupyterLab, and some basic programming skills in Python.

For simulating your own realizations, use (and possibly rename) the files beginning with *template*. Make a copy and rename them keeping the same extension and give them you own names. Then you'll be able to personalize a case with your own numbers and start experimenting with ARP Lab.

# Exploring the effect of coordinating assets allocations with ARP Lab

This notebook explores the effects of coordinating assets between savings accounts and spouses.

### Just some Python module bookkeeping
The following commands are bookkeeping calls that load the required module and set the program to display verbose informational messages as it runs. It also makes sure that all graphs are properly displayed within the jupyter interface.

These commands need to be at the beginning of any ARP Lab notebook.

In [ ]:
%matplotlib inline
# import arp

import importlib
arp = importlib.import_module('arp')
importlib.reload(arp)

arp.setVerbose(True)

## Initializing the realization

We consider Mary and John, an average couple that just retired.

In [ ]:
plan1 = arp.Plan(YOB=[1961, 1964], expectancy=[85, 88])

## Specify account balances and spousal beneficiaries


Mary and John have the following assets. They leave all their assets to the surviving spouse.

In [ ]:
plan1.setAssetBalances(taxable=[50000, 40000], 
                      taxDeferred=[1000000, 400000],
                      taxFree=[80000, 40000],
                      beneficiary=[1, 1])

## There must be a plan for savings

The couple has a few Roth conversions planned and a few more years to work. Open the Excel file to see the details. John is also planning a few Roth conversions.

In [ ]:
plan1.readContributions('mary+john.xlsx')

## What about anticipated fixed income?

Mary and John have no pension and average social security benefits.

In [ ]:
plan1.setPension([0, 0], [65, 65])
plan1.setSocialSecurity([30000, 26000], [70, 67])

## How much net annual income is desirable at retirement?

Mary and John desire an annual income of \\$80k.

In [ ]:
plan1.setDesiredIncome(80000, 'smile')

## How to withdraw from spousal accounts?

Rely on ARP Lab for avoiding individual funds exhaustion.

In [ ]:
plan1.setSpousalSplit('auto')

## Specify rates of return and inflation rate

Let's try coordination with a challenging year.

In [ ]:
plan1.setRates('historical', 1969)

## What are current and future assets allocations?

#### Our first per account, per individual allocation strategy.

In [ ]:
plan1.setInitialAR(taxable=[[60, 30, 5, 5], [60, 30, 5, 5]],
                  taxDeferred=[[60, 30, 5, 5], [60, 30, 5, 5]],
                  taxFree=[[60, 30, 5, 5], [60, 30, 5, 5]])

plan1.setFinalAR(taxable=[[50, 30, 5, 15], [50, 30, 5, 15]],
                taxDeferred=[[50, 30, 5, 15], [50, 30, 5, 15]],
                taxFree=[[50, 30, 5, 15], [50, 30, 5, 15]])

plan1.interpolateAR('s-curve')
plan1.showAllocations()

#### Coordinating between accounts

In [ ]:
plan2 = arp.clone(plan1)
# For coordination taking place for each individual separately.
plan2.setCoordinatedAR(initial=[[60, 30, 5, 5], [60, 30, 5, 5]], final=[[50, 30, 5, 15], [50, 30, 5, 15]])

# An interpolation must follow either call for generating the data for all the years of the simulation.
plan2.interpolateAR('linear')
plan2.showAllocations()

#### Coordinating between accounts and spouses
For a single individual, `plan2` and `plan3` would lead to the same answer.

In [ ]:
plan3 = arp.clone(plan1)

# For coordinating both spousal accounts simultaneously.
plan3.setCoordinatedAR(initial=[60, 30, 5, 5], final=[50, 30, 5, 15])

# An interpolation must follow either call for generating the data for all the years of the simulation.
plan3.interpolateAR('s-curve')
plan3.showAllocations()

## Generating the outcome of these scenarios
We're now ready to run the single instance of each scenario!


In [ ]:
arp.setVerbose(False)
plan1.run();
plan2.run();
plan3.run();

# Analysis

### Show some of the relevant info over the years

In [ ]:
for plan in [plan1, plan2, plan3]:
    plan.showNetIncome();

for plan in [plan1, plan2, plan3]:
    plan.showSources();

for plan in [plan1, plan2, plan3]:
    plan.showAccounts();


In [ ]:
for plan in [plan1, plan2, plan3]:
    plan.showGrossIncome();

for plan in [plan1, plan2, plan3]:
    plan.showTaxes();
  

In [ ]:
for plan in [plan1, plan2, plan3]:
    plan.estate(35)
    plan.showAssetsAllocations()
  

### Asset allocation coordination strategy is able to generate about 10% more in assets at the end of their life!
And this is done while maintaining the same overall percentage of assets allocations. This benefit is overlooked when one strictly looks at tax benefits of a Roth conversion. The reality is more complex and involves RMD's, tax that the heirs pay on the tax-deferred portion of the final assets, and the greater performance achieved in tax-free accounts. 